In [ ]:
import pandas as pd
import numpy as np 
from google.cloud import bigquery
from lifelines import CoxPHFitter  

from utils import read_and_render
from UKBInfoMapper import UKBInfoMapper
from utils_preprocessing import process_categorical_type_data, process_multi_categorical_type_data,process_date_type_data

In [ ]:
client = bigquery.Client()
path = 'sql/sql_file.sql'
query = read_and_render(path)
result = client.query(query).to_dataframe()
#120107 missing
result.columns = result.columns + '_'

In [ ]:
mapper_info = UKBInfoMapper()

In [ ]:
result = mapper_info._rename_columns_on_data(df=result,original_col="field_id",new_col="field_name")

In [ ]:
result = process_date_type_data(result,'DateOfAttendingAssessmentCentre')
list_columns_date = ['DateLostToFollow-Up','DateOfDeath','outcome_date_','max_date_']
for col in list_columns_date:
    result = process_date_type_data(result,col)
    result[col] = (result[col] - result['DateOfAttendingAssessmentCentre'])/pd.to_timedelta('365 days')
result['survival_time'] = np.nanmin(result[list_columns_date],1)

In [ ]:
result = result.drop( ['DateLostToFollow-Up', 'DateOfAttendingAssessmentCentre','DateOfDeath','outcome_date_','max_date_'], axis=1)
FILL_0 = ['TobaccoSmoking'] 
result[FILL_0] = result[FILL_0].fillna(114)
for col in result.columns:
    if col not in ['_eid_']:
        try:
            value_type = mapper_info.df[mapper_info.df.field_name==col].value_type.values[0]
            if value_type == 11:
                result[col] = result[col].fillna(result[col].mean())
            else:
                result[col] = result[col].fillna(result[col].mode())
        except:
            pass

In [ ]:
drop_low_count_rows = ["SeverityOfManic/IrritableEpisodes", "EverUnenthusiastic/DisinterestedForAWholeWeek", "EverDepressedForAWholeWeek", "FinancialSituationSatisfaction"]
result = result.drop(drop_low_count_rows, axis=1)
result = result.dropna()

In [ ]:
result = mapper_info._rename_columns_on_data(df=result,original_col="field_name",new_col="field_id")

In [ ]:
mapper_info.df = mapper_info.df[mapper_info.df.field_id.isin(result.columns)]
list_col_21 = list(set(mapper_info.df[mapper_info.df.value_type == 21].field_id.tolist() ))
for col in list_col_21:
    result = process_categorical_type_data(result, col)
list_col_22 = list(set(mapper_info.df[mapper_info.df.value_type == 22].field_id.tolist()))
for col in list_col_22:
    result = process_multi_categorical_type_data(result, col)

In [ ]:
result = mapper_info._rename_columns_on_data(df=result,original_col="field_id_processed",new_col="field_name_processed")

In [ ]:
result = result.set_index('_eid_')

In [12]:
model = CoxPHFitter(penalizer=0.1)
model.fit(result, "survival_time", "outcome_")

In [ ]:
model.summary = model.summary.loc[model.summary.p<0.05].sort_values('coef')
model.print_summary()